In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##데이터준비

###데이터 불러오기

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# 상품분류통합된 구매정보 데이터 불러오기
npurch = pd.read_csv('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/dataset/create_feature/df_purchase_trl.csv',
                    encoding='cp949', chunksize = 1000000) #100만개 단위로 끊기, 실제 transaction 데이터
chunk = list(npurch)

print(len(chunk)) # 몇개 chunk인지 출력

npurch = pd.concat(chunk)  # 데이터 넣기
npurch['구매일자'] = pd.to_datetime(npurch['구매일자'],format='%Y-%m-%d')
npurch.shape

29


(28593030, 13)

In [ ]:
npurch.head()

,제휴사,고객번호,구매일자,구매시간,구매금액,중분류명,점포,거주지역,거주지,점포지역,월,년도,4분기
0,B,17218,2014-02-22,20,2420,N음료류,B44,100.0,경기,경기,2,2014,1
1,B,17218,2014-02-22,20,1070,주류,B44,100.0,경기,경기,2,2014,1
2,B,17218,2014-02-22,20,8060,주류,B44,100.0,경기,경기,2,2014,1
3,B,17218,2014-02-22,20,6000,일용잡화,B44,100.0,경기,경기,2,2014,1
4,B,17218,2015-10-23,18,2850,주방용품,B44,100.0,경기,경기,10,2015,4


In [ ]:
for i in ['A', 'B', 'C', 'D']:
  mom_df = npurch[npurch['제휴사'] == i]
  mom_df.drop(['제휴사', '구매시간', '점포', '거주지역', '거주지', '점포지역', '월', '년도', '4분기'], axis=1, inplace=True)

  df_final = pd.merge(mom_df, quan_grade(mom_df, '2014-01-01', '2014-03-31', 'grade14_1'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-04-01', '2014-06-30', 'grade14_2'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-07-01', '2014-09-30', 'grade14_3'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-10-01', '2014-12-31', 'grade14_4'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2015-01-01', '2015-03-31', 'grade15_1'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2015-04-01', '2015-06-30', 'grade15_2'), on='고객번호', how='right')

  df_final.iloc[:, 4:] = df_final.iloc[:, 4:].fillna(0)

  df_final['fluctuation'] = (df_final['grade14_1'] - df_final['grade14_2']) + \
                            (df_final['grade14_2'] - df_final['grade14_3']) + \
                            (df_final['grade14_3'] - df_final['grade14_4']) + \
                            (df_final['grade14_4'] - df_final['grade15_1']) + \
                            (df_final['grade15_1'] - df_final['grade15_2'])

  grouped = df_final.groupby(['고객번호', 'fluctuation'], as_index=False)[['구매금액']].sum()

  df_final = pd.merge(basedf1, grouped, on='고객번호', how='left')[['고객번호', 'fluctuation']]
  df_final.shape

  # if len(li[i].split('/')) > 1:
  #   x, y = li[i].split('/')
  #   df_final.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./fluctuation_{x+y}.csv', index=False)
  # else:
  df_final.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./fluctuation_{i}.csv', index=False)

In [ ]:
for i in ['A', 'B', 'C', 'D']:
  mom_df = npurch[npurch['제휴사'] == i]

  df_final = pd.merge(mom_df, quan_grade(mom_df, '2014-04-01', '2014-06-30', 'grade14_2'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-07-01', '2014-09-30', 'grade14_3'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-10-01', '2014-12-31', 'grade14_4'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2015-01-01', '2015-03-31', 'grade15_1'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2015-04-01', '2015-06-30', 'grade15_2'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2015-07-01', '2015-09-30', 'grade15_3'), on='고객번호', how='right')

  df_final.iloc[:, 4:] = df_final.iloc[:, 4:].fillna(0)

  df_final['fluctuation'] = (df_final['grade14_2'] - df_final['grade14_3']) + \
                            (df_final['grade14_3'] - df_final['grade14_4']) + \
                            (df_final['grade14_4'] - df_final['grade15_1']) + \
                            (df_final['grade15_1'] - df_final['grade15_2']) + \
                            (df_final['grade15_2'] - df_final['grade15_3'])

  grouped = df_final.groupby(['고객번호', 'fluctuation'], as_index=False)[['구매금액']].sum()

  df_final = pd.merge(basedf2, grouped, on='고객번호', how='left')[['고객번호', 'fluctuation']]
  df_final.shape
  
  df_final.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./test_fluctuation_{i}.csv', index=False)

##quan_grade 함수

In [ ]:
def quan_grade(mom_df, quan_start, quan_end, col_name):

  df = mom_df[mom_df['구매일자'].between(quan_start, quan_end)]

  # gorupby
  df = df.groupby(['고객번호'], as_index=False)[['구매금액']].sum()

  # pd.cut
  p_li = [0.04, 0.11, 0.23, 0.40, 0.60, 0.77, 0.89, 0.96] # 퍼센트
  bins = [0] + [df['구매금액'].quantile(q=p_li[i]) for i in range(8)] + [df['구매금액'].quantile(1)] # 등급별 기준값
  labels = [1, 2, 3, 4, 5, 6, 7, 8, 9] # 등급

  df[col_name] = pd.cut(df['구매금액'], bins=bins, labels=labels)
  df[col_name] = pd.to_numeric(df[col_name])

  # unvaluable column out
  df.drop('구매금액', axis=1, inplace=True)

  return df

##피처1. 고객 분기별 중분류상품 총구매금액 등급 -> 증감지수

###사전작업

In [ ]:
# 중분류명 담은 리스트 생성
li = npurch['중분류명'].unique()
print(li)

['N음료류' '주류' '일용잡화' '주방용품' 'N스낵류' '가공식품' '생활잡화' '농산물' '스포츠' '육류' '차/커피'
 '화장품' 'N냉동류' '섬유잡화' '젓갈/반찬' '식기' '기타' '액세서리' '란제리/내의' 'N냉장류' '건강식품' '수산품'
 '캐주얼' '구두' '축산가공' 'N푸드코트' '전자/전기' '문구/팬시' '기타의류' '남성 트랜디' '아동' 'N자동차용품'
 '침구/수예' 'N도서/음반' '가구' '소형가전' '준보석/시계' '명품' '남성캐주얼' '골프용품' '피혁잡화' '남성정장'
 '셔츠' '통신/컴퓨터' '트래디셔널' '보석' '커리어' '주방가전' '시티웨어' '디자이너' '모피/피혁' '대형가전' '교복'
 '악기' '블라우스' '웨딩']


In [ ]:
# 고객번호 drop 없이 출력하기 위해 틀 생성
basedf1 = pd.read_csv('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/dataset/create_feature/lmembers_features_통합(train또는1_6분기데이터)_ 0.001로 바꿈.csv')['고객번호']

In [ ]:
basedf1

0            1
1            2
2            3
3            4
4            5
         ...  
19367    19379
19368    19380
19369    19381
19370    19382
19371    19383
Name: 고객번호, Length: 19372, dtype: int64

In [ ]:
# 고객번호 drop 없이 출력하기 위해 틀 생성
basedf2 = pd.read_csv('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/dataset/create_feature/lmembers_features_통합(test또는2_7분기데이터)_0.001로바꿈.csv')[['고객번호']]

In [ ]:
basedf2

,고객번호
0,1
1,2
2,3
3,4
4,5
...,...
19366,19379
19367,19380
19368,19381
19369,19382


###테스트

####중분류별 증감지수

In [ ]:
# 테스트

i = 0
mom_df = npurch[npurch['중분류명'] == li[i]]
mom_df.drop(['제휴사', '구매시간', '점포', '거주지역', '거주지', '점포지역', '월', '년도', '4분기'], axis=1, inplace=True)

df_final = pd.merge(mom_df, quan_grade(mom_df, '2014-01-01', '2014-03-31', 'grade14_1'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(mom_df, '2014-04-01', '2014-06-30', 'grade14_2'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(mom_df, '2014-07-01', '2014-09-30', 'grade14_3'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(mom_df, '2014-10-01', '2014-12-31', 'grade14_4'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(mom_df, '2015-01-01', '2015-03-31', 'grade15_1'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(mom_df, '2015-04-01', '2015-06-30', 'grade15_2'), on='고객번호', how='right')

df_final.iloc[:, 4:] = df_final.iloc[:, 4:].fillna(0)

df_final['fluctuation'] = (df_final['grade14_1'] - df_final['grade14_2']) + \
                          (df_final['grade14_2'] - df_final['grade14_3']) + \
                          (df_final['grade14_3'] - df_final['grade14_4']) + \
                          (df_final['grade14_4'] - df_final['grade15_1']) + \
                          (df_final['grade15_1'] - df_final['grade15_2'])

grouped = df_final.groupby(['고객번호', 'fluctuation'], as_index=False)[['구매금액']].sum()

In [ ]:
# 고객번호 drop 없이 merge
df_final = pd.merge(basedf1, grouped, on='고객번호', how='left')[['고객번호', 'fluctuation']]
df_final.shape

(19327, 2)

In [ ]:
# 파일로 저장
# df_final.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./fluctuation_{li[i]}.csv', index=False)

- 문제점
1. 고객의 구매이력이 없는 분기에는 0등급인데, 다음 분기에 영향을 크게 미쳐서 결국 fluctuation 값이 심각하게 떨어진다.
2. 해당 중분류상품을 구매한 이력이 없는 경우 fluctuation이 결측치이다.
3. 원본 데이터프레임에는 1번 고객의 구매금액이 있다. 그러나 groupby 후에는 구매금액이 0으로 나온다.

In [ ]:
mom_df[mom_df['고객번호']==1]

,고객번호,구매일자,구매금액,중분류명
8219557,1,2015-04-25,6600,N음료류
8219587,1,2014-10-09,8400,N음료류
8220206,1,2014-08-13,11280,N음료류
8220385,1,2015-01-08,11280,N음료류


In [ ]:
grouped.head()

,고객번호,fluctuation,구매금액
0,1,-3.0,0.0
1,3,2.0,251490.0
2,6,8.0,605290.0
3,9,-7.0,0.0
4,10,1.0,56140.0


#### for문 반복작업

1부터 55까지 바꾸며 돌려서 파일 저장.

In [ ]:
len(li)

56

In [ ]:
# 중분류명에 '/'로 Error 피하기 위한 작업
x, y = li[10].split('/')
print(x+y)

차커피


In [ ]:
li

array(['N음료류', '주류', '일용잡화', '주방용품', 'N스낵류', '가공식품', '생활잡화', '농산물', '스포츠',
       '육류', '차/커피', '화장품', 'N냉동류', '섬유잡화', '젓갈/반찬', '식기', '기타', '액세서리',
       '란제리/내의', 'N냉장류', '건강식품', '수산품', '캐주얼', '구두', '축산가공', 'N푸드코트',
       '전자/전기', '문구/팬시', '기타의류', '남성 트랜디', '아동', 'N자동차용품', '침구/수예',
       'N도서/음반', '가구', '소형가전', '준보석/시계', '명품', '남성캐주얼', '골프용품', '피혁잡화',
       '남성정장', '셔츠', '통신/컴퓨터', '트래디셔널', '보석', '커리어', '주방가전', '시티웨어',
       '디자이너', '모피/피혁', '대형가전', '교복', '악기', '블라우스', '웨딩'], dtype=object)

In [ ]:
for i in range(52, len(li)):
  mom_df = npurch[npurch['중분류명'] == li[i]]
  mom_df.drop(['제휴사', '구매시간', '점포', '거주지역', '거주지', '점포지역', '월', '년도', '4분기'], axis=1, inplace=True)

  df_final = pd.merge(mom_df, quan_grade(mom_df, '2014-01-01', '2014-03-31', 'grade14_1'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-04-01', '2014-06-30', 'grade14_2'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-07-01', '2014-09-30', 'grade14_3'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-10-01', '2014-12-31', 'grade14_4'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2015-01-01', '2015-03-31', 'grade15_1'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2015-04-01', '2015-06-30', 'grade15_2'), on='고객번호', how='right')

  df_final.iloc[:, 4:] = df_final.iloc[:, 4:].fillna(0)

  df_final['fluctuation'] = (df_final['grade14_1'] - df_final['grade14_2']) + \
                            (df_final['grade14_2'] - df_final['grade14_3']) + \
                            (df_final['grade14_3'] - df_final['grade14_4']) + \
                            (df_final['grade14_4'] - df_final['grade15_1']) + \
                            (df_final['grade15_1'] - df_final['grade15_2'])

  grouped = df_final.groupby(['고객번호', 'fluctuation'], as_index=False)[['구매금액']].sum()

  df_final = pd.merge(basedf1, grouped, on='고객번호', how='left')[['고객번호', 'fluctuation']]
  df_final.shape

  if len(li[i].split('/')) > 1:
    x, y = li[i].split('/')
    # df_final.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./fluctuation_{x+y}.csv', index=False)
  else:
    # df_final.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./fluctuation_{li[i]}.csv', index=False)

####미처리된 작업

In [ ]:
i # for문 멈춘 지점
# ValueError: Bin edges must be unique: array([ 0., nan, nan, nan, nan, nan, nan, nan, nan, nan]).
# You can drop duplicate edges by setting the 'duplicates' kwarg

# ['교복', '악기', '블라우스', '웨딩'] 작업 불가. 값이 없나? 아하, 가격대를 나누었는데 그만큼의 값이 안 나오는 건가?

52

In [ ]:
npurch[npurch['중분류명'] == '교복']['구매금액'].unique() # 아닌데..

array([ 16000, 409000, 183000, 105000, 353000, 250000, 103000, 278000,
        63000, 347500, 168000, 160000,  85000, 475000,  58000, 249000,
       144000, 320000, 350000, 128000, 165000,  66000,  56000, 180000,
       420000, 190000,  25000, 430000, 217000,  44000, 175000,  23000,
       352000,  94000,  27000, 134000, 248000,  65000, 399000, 413000,
       223000, 421000, 194000, 302500, 140000, 193000,  50000, 322000,
       245000, 302000,  32000, 355500,  35000, 229000, 410800, 479000,
         5000, 481000,  60000,  42000, 158000,  30000, 106000, 383000,
       354000, 192000, 381000, 224000, 408000, 357000,  96000, 149000,
       349000, 170000, 283000,  53000, 120000, 280000, 274000, 255000,
       281000,   6000, 102000,  40000, 382000, 372400, 429000,  48000,
       253000,  88000,  54000, 179000,  33000, 188000, 319000, 148000,
       130000,  24000, 221000, 367000,  90000, 107000, 100000, 239000,
       306000, 206000, 438000, 358000,  80000, 121000,  55000,  20000,
      

In [ ]:
import glob

# file_name = glob.glob('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./*.csv')
print(len(file_name))
file_name

45


['/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/매출대비구매비율.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_N음료류.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_주류.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_일용잡화.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_주방용품.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_N스낵류.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_가공식품.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_생활잡화.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_농산물.csv',
 '/content/d

- 재시도

In [ ]:
li.sort()
print(li) # 디렉토리 내 저장된 파일들과 비교하여 미처리된 중분류명 확인
print('-----')
li1 = ['N도서/음반', '란제리/내의', '모피/피혁', '문구/팬시', '전자/전기', '젓갈/반찬',
      '준보석/시계', '침구/수예', '통신/컴퓨터']
li2 = ['교복', '블라우스', '웨딩']
print(li1)
print(li2)

['N냉동류' 'N냉장류' 'N도서/음반' 'N스낵류' 'N음료류' 'N자동차용품' 'N푸드코트' '가공식품' '가구' '건강식품'
 '골프용품' '교복' '구두' '기타' '기타의류' '남성 트랜디' '남성정장' '남성캐주얼' '농산물' '대형가전' '디자이너'
 '란제리/내의' '명품' '모피/피혁' '문구/팬시' '보석' '블라우스' '생활잡화' '섬유잡화' '셔츠' '소형가전' '수산품'
 '스포츠' '시티웨어' '식기' '아동' '악기' '액세서리' '웨딩' '육류' '일용잡화' '전자/전기' '젓갈/반찬' '주류'
 '주방가전' '주방용품' '준보석/시계' '차/커피' '축산가공' '침구/수예' '캐주얼' '커리어' '통신/컴퓨터' '트래디셔널'
 '피혁잡화' '화장품']
-----
['N도서/음반', '란제리/내의', '모피/피혁', '문구/팬시', '전자/전기', '젓갈/반찬', '준보석/시계', '침구/수예', '통신/컴퓨터']
['교복', '블라우스', '웨딩']


li1 처리

In [ ]:
for i in range(len(li1)):
  mom_df = npurch[npurch['중분류명'] == li[i]]
  mom_df.drop(['제휴사', '구매시간', '점포', '거주지역', '거주지', '점포지역', '월', '년도', '4분기'], axis=1, inplace=True)

  df_final = pd.merge(mom_df, quan_grade(mom_df, '2014-01-01', '2014-03-31', 'grade14_1'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-04-01', '2014-06-30', 'grade14_2'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-07-01', '2014-09-30', 'grade14_3'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-10-01', '2014-12-31', 'grade14_4'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2015-01-01', '2015-03-31', 'grade15_1'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2015-04-01', '2015-06-30', 'grade15_2'), on='고객번호', how='right')

  df_final.iloc[:, 4:] = df_final.iloc[:, 4:].fillna(0)

  df_final['fluctuation'] = (df_final['grade14_1'] - df_final['grade14_2']) + \
                            (df_final['grade14_2'] - df_final['grade14_3']) + \
                            (df_final['grade14_3'] - df_final['grade14_4']) + \
                            (df_final['grade14_4'] - df_final['grade15_1']) + \
                            (df_final['grade15_1'] - df_final['grade15_2'])

  grouped = df_final.groupby(['고객번호', 'fluctuation'], as_index=False)[['구매금액']].sum()

  df_final = pd.merge(basedf1, grouped, on='고객번호', how='left')[['고객번호', 'fluctuation']]
  df_final.shape

  x, y = li1[i].split('/')
  df_final.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_{x+y}.csv', index=False)

  # if len(li[i].split('/')) > 1:
    # x, y = li[i].split('/')
    # df_final.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./fluctuation_{x+y}.csv', index=False)
  # else:
  #   df_final.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./fluctuation_{li[i]}.csv', index=False)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


li2 = ['교복', '블라우스', '웨딩']
- 데이터가 현저히 부족해서 유의미한 피처가 되지 못할 것으로 보임.

In [ ]:
train_purch = npurch[npurch['구매일자'].between('2014-01-01', '2015-06-30')]

데이터 존재여부 확인
- '교복' 이력 유
- '블라우스' 이력 무
- '웨딩' 이력 유

In [ ]:
temp = train_purch[train_purch['중분류명'] == '웨딩']

In [ ]:
temp

,제휴사,고객번호,구매일자,구매시간,구매금액,중분류명,점포,거주지역,거주지,점포지역,월,년도,4분기
9703139,A,9163,2015-05-29,18,1895000,웨딩,A2,55.0,서울,서울,5,2015,2
9704149,A,9163,2015-05-01,14,2808000,웨딩,A2,55.0,서울,서울,5,2015,2
16466136,A,16940,2014-08-11,13,55100,웨딩,A12,210.0,인천,서울,8,2014,3


해당 분기에 구매이력이 없으므로 에러가 난 것.

- 교복

In [ ]:
mom_df = npurch[npurch['중분류명'] == '교복']
mom_df.drop(['제휴사', '구매시간', '점포', '거주지역', '거주지', '점포지역', '월', '년도', '4분기'], axis=1, inplace=True)

df_final = pd.merge(mom_df, quan_grade(mom_df, '2014-01-01', '2014-03-31', 'grade14_1'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(mom_df, '2014-04-01', '2014-06-30', 'grade14_2'), on='고객번호', how='right')
# df_final = pd.merge(df_final, quan_grade(mom_df, '2014-07-01', '2014-09-30', 'grade14_3'), on='고객번호', how='right')
# df_final = pd.merge(df_final, quan_grade(mom_df, '2014-10-01', '2014-12-31', 'grade14_4'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(mom_df, '2015-01-01', '2015-03-31', 'grade15_1'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(mom_df, '2015-04-01', '2015-06-30', 'grade15_2'), on='고객번호', how='right')

df_final.iloc[:, 4:] = df_final.iloc[:, 4:].fillna(0)

'''df_final['fluctuation'] = (df_final['grade14_1'] - df_final['grade14_2']) + \
                          (df_final['grade14_2'] - df_final['grade14_3']) + \
                          (df_final['grade14_3'] - df_final['grade14_4']) + \
                          (df_final['grade14_4'] - df_final['grade15_1']) + \
                          (df_final['grade15_1'] - df_final['grade15_2'])'''

df_final['fluctuation'] = (df_final['grade14_1'] - df_final['grade14_2']) + \
                          (df_final['grade14_2'] - df_final['grade15_1']) + \
                          (df_final['grade15_1'] - df_final['grade15_2'])

grouped = df_final.groupby(['고객번호', 'fluctuation'], as_index=False)[['구매금액']].sum()

df_final = pd.merge(basedf1, grouped, on='고객번호', how='left')[['고객번호', 'fluctuation']]
df_final.shape

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(19327, 2)

In [ ]:
# df_final.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./fluctuation_교복.csv', index=False)

- 웨딩

In [ ]:
mom_df = npurch[npurch['중분류명'] == '웨딩']
mom_df.drop(['제휴사', '구매시간', '점포', '거주지역', '거주지', '점포지역', '월', '년도', '4분기'], axis=1, inplace=True)

# df_final = pd.merge(mom_df, quan_grade(mom_df, '2014-01-01', '2014-03-31', 'grade14_1'), on='고객번호', how='right')
# df_final = pd.merge(mom_df, quan_grade(mom_df, '2014-04-01', '2014-06-30', 'grade14_2'), on='고객번호', how='right')
df_final = pd.merge(mom_df, quan_grade(mom_df, '2014-07-01', '2014-09-30', 'grade14_3'), on='고객번호', how='right')
# df_final = pd.merge(mom_df, quan_grade(mom_df, '2014-10-01', '2014-12-31', 'grade14_4'), on='고객번호', how='right')
# df_final = pd.merge(mom_df, quan_grade(mom_df, '2015-01-01', '2015-03-31', 'grade15_1'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(mom_df, '2015-04-01', '2015-06-30', 'grade15_2'), on='고객번호', how='right')

df_final.iloc[:, 4:] = df_final.iloc[:, 4:].fillna(0)

# df_final['fluctuation'] = (df_final['grade14_1'] - df_final['grade14_2']) + \
#                           (df_final['grade14_2'] - df_final['grade14_3']) + \
#                           (df_final['grade14_3'] - df_final['grade14_4']) + \
#                           (df_final['grade14_4'] - df_final['grade15_1']) + \
#                           (df_final['grade15_1'] - df_final['grade15_2'])

df_final['fluctuation'] = (df_final['grade14_3'] - df_final['grade15_2'])

grouped = df_final.groupby(['고객번호', 'fluctuation'], as_index=False)[['구매금액']].sum()

df_final = pd.merge(basedf1, grouped, on='고객번호', how='left')[['고객번호', 'fluctuation']]
df_final.shape

# df_final.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./fluctuation_웨딩.csv', index=False)

###최종작업

In [ ]:
# 중분류명 담은 리스트 생성
li = npurch['중분류명'].unique()
print(li)
print(len(li))

['N음료류' '주류' '일용잡화' '주방용품' 'N스낵류' '가공식품' '생활잡화' '농산물' '스포츠' '육류' '차/커피'
 '화장품' 'N냉동류' '섬유잡화' '젓갈/반찬' '식기' '기타' '액세서리' '란제리/내의' 'N냉장류' '건강식품' '수산품'
 '캐주얼' '구두' '축산가공' 'N푸드코트' '전자/전기' '문구/팬시' '기타의류' '남성 트랜디' '아동' 'N자동차용품'
 '침구/수예' 'N도서/음반' '가구' '소형가전' '준보석/시계' '명품' '남성캐주얼' '골프용품' '피혁잡화' '남성정장'
 '셔츠' '통신/컴퓨터' '트래디셔널' '보석' '커리어' '주방가전' '시티웨어' '디자이너' '모피/피혁' '대형가전' '교복'
 '악기' '블라우스' '웨딩']
56


In [ ]:
# 데이터가 부족한 ['교복', '블라우스', '웨딩'] 제거
removed = np.delete(li, [[52, 54, 55]])

In [ ]:
len(removed)

53

###14년1분기~15년2분기 작업

In [ ]:
temp = npurch[npurch['구매일자'].between('2014-01-01', '2015-06-30')]
temp = temp.drop(['제휴사', '구매시간', '점포',
                    '거주지역', '거주지', '점포지역', '월', '년도', '4분기'], axis=1)

In [ ]:
for i in range(len(removed)):
  mom_df = temp[temp['중분류명'] == removed[i]]

  df_final = pd.merge(mom_df, quan_grade(mom_df, '2014-01-01', '2014-03-31', 'grade14_1'), on='고객번호', how='left')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-04-01', '2014-06-30', 'grade14_2'), on='고객번호', how='left')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-07-01', '2014-09-30', 'grade14_3'), on='고객번호', how='left')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-10-01', '2014-12-31', 'grade14_4'), on='고객번호', how='left')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2015-01-01', '2015-03-31', 'grade15_1'), on='고객번호', how='left')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2015-04-01', '2015-06-30', 'grade15_2'), on='고객번호', how='left')

  df_final.iloc[:, 4:] = df_final.iloc[:, 4:].fillna(0)

  df_final['fluctuation'] = (df_final['grade14_1'] - df_final['grade14_2']) + \
                            (df_final['grade14_2'] - df_final['grade14_3']) + \
                            (df_final['grade14_3'] - df_final['grade14_4']) + \
                            (df_final['grade14_4'] - df_final['grade15_1']) + \
                            (df_final['grade15_1'] - df_final['grade15_2'])

  grouped = df_final.groupby(['고객번호', 'fluctuation'], as_index=False)[['구매금액']].sum()
  print(grouped.shape)

  if len(removed[i].split('/')) > 1:
    x, y = removed[i].split('/')
    # grouped.to_csv(f/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./train_fluctuation_{x+y}.csv', index=False)
  else:
    # grouped.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./train_fluctuation_{removed[i]}.csv', index=False)

(17284, 3)
(18442, 3)
(19003, 3)
(18617, 3)
(17855, 3)
(19328, 3)
(18884, 3)
(19189, 3)
(18074, 3)
(18820, 3)
(19034, 3)
(19027, 3)
(15818, 3)
(15779, 3)
(15898, 3)
(14664, 3)
(12818, 3)
(13039, 3)
(15015, 3)
(15017, 3)
(14183, 3)
(18756, 3)
(17473, 3)
(12137, 3)
(12544, 3)
(11888, 3)
(9727, 3)
(13485, 3)
(11674, 3)
(7451, 3)
(13437, 3)
(6575, 3)
(9217, 3)
(7203, 3)
(9208, 3)
(2805, 3)
(2542, 3)
(7277, 3)
(5697, 3)
(5904, 3)
(10215, 3)
(3941, 3)
(6511, 3)
(3795, 3)
(8419, 3)
(2152, 3)
(5424, 3)
(6629, 3)
(4123, 3)
(2859, 3)
(3795, 3)
(2236, 3)
(239, 3)


####하나의 데이터프레임으로 합치기

In [ ]:
# 파일명 리스트 생성
import glob
# file_name_list = glob.glob('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./*.csv')
file_name_list

['/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/중분류작업/train/train_fluctuation_일용잡화.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/중분류작업/train/train_fluctuation_주류.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/중분류작업/train/train_fluctuation_주방용품.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/중분류작업/train/train_fluctuation_N스낵류.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/중분류작업/train/train_fluctuation_가공식품.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/중분류작업/train/train_fluctuation_생활잡화.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/중분류작업/train/train_fluctuation_농산물.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/중분류작업/train/train_fluctuation_스포츠

In [ ]:
len(file_name_list)

52

In [ ]:
file_name_list.pop(53) # .pop() 에 인덱스를 인수로 넣어 필요 없는 파일 제거
file_name_list.pop(53)
file_name_list

['/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/train/train_fluctuation_N음료류.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/train/train_fluctuation_주류.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/train/train_fluctuation_일용잡화.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/train/train_fluctuation_주방용품.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/train/train_fluctuation_N스낵류.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/train/train_fluctuation_가공식품.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/train/train_fluctuation_생활잡화.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/train/train_fluctuation_농산물.csv',
 '/content/driv

In [ ]:
# 정규표현식 이용하여 중분류명 추출
import re
p = re.compile('(?<=train_fluctuation_).+(?=.csv)')
m = re.search(p, str(file_name_list[1]))
m.group()

'주류'

In [ ]:
file_name_list[0]

'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/중분류작업/test/test_fluctuation_N냉동류.csv'

In [ ]:
pd.merge(basedf1, df[['고객번호','fluctuation']], on='고객번호', how='left')

,고객번호,fluctuation
0,1,7.0
1,2,0.0
2,3,2.0
3,4,0.0
4,5,5.0
...,...,...
19367,19379,-7.0
19368,19380,-6.0
19369,19381,-4.0
19370,19382,-2.0


In [ ]:
# 고객번호 데이터프레임에 중분류별 증감지수 열 추가

for file_name in file_name_list:
  df = pd.read_csv(file_name)
  m = re.search(p, file_name)
  name = 'f_' + m.group()
  merge_df = pd.merge(basedf1, df[['고객번호','fluctuation']], on='고객번호', how='left')  
  basedf1[name] = merge_df['fluctuation']
  print(name, merge_df.shape)
  print(merge_df.isna().sum())

f_N냉동류 (19372, 2)
고객번호              0
fluctuation    3383
dtype: int64
f_N냉장류 (19372, 3)
고객번호              0
f_N냉동류    3383
fluctuation    4178
dtype: int64
f_N도서음반 (19372, 4)
고객번호               0
f_N냉동류     3383
f_N냉장류     4178
fluctuation    12177
dtype: int64
f_N스낵류 (19372, 5)
고객번호                 0
f_N냉동류       3383
f_N냉장류       4178
f_N도서음반    12177
fluctuation       1495
dtype: int64
f_N음료류 (19372, 6)
고객번호                 0
f_N냉동류       3383
f_N냉장류       4178
f_N도서음반    12177
f_N스낵류        1495
fluctuation       2000
dtype: int64
f_N자동차용품 (19372, 7)
고객번호                 0
f_N냉동류       3383
f_N냉장류       4178
f_N도서음반    12177
f_N스낵류        1495
f_N음료류        2000
fluctuation      12670
dtype: int64
f_N푸드코트 (19372, 8)
고객번호                    0
f_N냉동류          3383
f_N냉장류          4178
f_N도서음반       12177
f_N스낵류           1495
f_N음료류           2000
f_

In [ ]:
basedf1.head()

,고객번호,f_N냉동류,f_N냉장류,f_N도서음반,f_N스낵류,f_N음료류,f_N자동차용품,f_N푸드코트,f_가공식품,f_가구,...,f_준보석시계,f_차커피,f_축산가공,f_침구수예,f_캐주얼,f_커리어,f_통신컴퓨터,f_트래디셔널,f_피혁잡화,f_화장품
0,1,0.0,0.0,NaN,0.0,0.0,NaN,0.0,-2.0,NaN,...,0.0,4.0,5.0,0.0,0.0,-4.0,0.0,-3.0,-5.0,7.0
1,2,NaN,NaN,NaN,NaN,-2.0,NaN,NaN,-1.0,NaN,...,NaN,3.0,0.0,-4.0,-7.0,-1.0,NaN,4.0,0.0,0.0
2,3,0.0,0.0,NaN,1.0,-1.0,NaN,NaN,1.0,NaN,...,NaN,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,4,8.0,6.0,NaN,0.0,-2.0,NaN,NaN,-1.0,-4.0,...,NaN,2.0,-1.0,-5.0,0.0,1.0,NaN,NaN,NaN,0.0
4,5,NaN,NaN,NaN,0.0,NaN,NaN,NaN,5.0,NaN,...,NaN,7.0,9.0,0.0,2.0,NaN,NaN,0.0,NaN,5.0


In [ ]:
basedf1.to_excel('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/dataset/create_feature/train_fluctuation_중분류별.xlsx',
                 encoding='utf-8-sig', index=False)

###14년2분기~15년3분기 작업

In [ ]:
temp = npurch[npurch['구매일자'].between('2014-04-01', '2015-09-30')]
temp = temp.drop(['제휴사', '구매시간', '점포',
                    '거주지역', '거주지', '점포지역', '월', '년도', '4분기'], axis=1)

In [ ]:
for i in range(len(removed)):
  mom_df = temp[temp['중분류명'] == removed[i]]

  df_final = pd.merge(mom_df, quan_grade(mom_df, '2014-04-01', '2014-06-30', 'grade14_2'), on='고객번호', how='left')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-07-01', '2014-09-30', 'grade14_3'), on='고객번호', how='left')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-10-01', '2014-12-31', 'grade14_4'), on='고객번호', how='left')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2015-01-01', '2015-03-31', 'grade15_1'), on='고객번호', how='left')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2015-04-01', '2015-06-30', 'grade15_2'), on='고객번호', how='left')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2015-07-01', '2015-09-30', 'grade15_3'), on='고객번호', how='left')

  df_final.iloc[:, 4:] = df_final.iloc[:, 4:].fillna(0)

  df_final['fluctuation'] = (df_final['grade14_2'] - df_final['grade14_3']) + \
                            (df_final['grade14_3'] - df_final['grade14_4']) + \
                            (df_final['grade14_4'] - df_final['grade15_1']) + \
                            (df_final['grade15_1'] - df_final['grade15_2']) + \
                            (df_final['grade15_2'] - df_final['grade15_3'])

  grouped = df_final.groupby(['고객번호', 'fluctuation'], as_index=False)[['구매금액']].sum()
  print(grouped.shape)
  
  if len(removed[i].split('/')) > 1:
    x, y = removed[i].split('/')
    # grouped.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./test_fluctuation_{x+y}.csv', index=False)
  else:
    # grouped.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./test_fluctuation_{removed[i]}.csv', index=False)

(17373, 3)
(18539, 3)
(19012, 3)
(18596, 3)
(17879, 3)
(19331, 3)
(18914, 3)
(19182, 3)
(18138, 3)
(18821, 3)
(19074, 3)
(19031, 3)
(15990, 3)
(16014, 3)
(16037, 3)
(14772, 3)
(13333, 3)
(13202, 3)
(15108, 3)
(15194, 3)
(14310, 3)
(18728, 3)
(17623, 3)
(12045, 3)
(13180, 3)
(12020, 3)
(10437, 3)
(13473, 3)
(11533, 3)
(6956, 3)
(13465, 3)
(6702, 3)
(9816, 3)
(7195, 3)
(8553, 3)
(2490, 3)
(2555, 3)
(7437, 3)
(6217, 3)
(5992, 3)
(10047, 3)
(4033, 3)
(6509, 3)
(3126, 3)
(8521, 3)
(2370, 3)
(5577, 3)
(6581, 3)
(4054, 3)
(2810, 3)
(3928, 3)
(2134, 3)
(209, 3)


In [ ]:
i

55

Error i = [52, 54, 55]

In [ ]:
li[[52, 54, 55]] # ['교복', '블라우스', '웨딩'] 데이터 부족

array(['교복', '블라우스', '웨딩'], dtype=object)

#### 하나의 데이터프레임으로 합치기

In [ ]:
# 파일명 리스트 생성
import glob
# file_name_list = glob.glob('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./*.csv')

# file_name_list.pop(0) # .pop() 에 인덱스를 인수로 넣어 필요 없는 파일 제거
# file_name_list.pop(53)

len(file_name_list)


53

In [ ]:
file_name_list.sort()

In [ ]:
file_name_list

['/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/중분류작업/test/test_fluctuation_N냉동류.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/중분류작업/test/test_fluctuation_N냉장류.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/중분류작업/test/test_fluctuation_N도서음반.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/중분류작업/test/test_fluctuation_N스낵류.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/중분류작업/test/test_fluctuation_N음료류.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/중분류작업/test/test_fluctuation_N자동차용품.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/중분류작업/test/test_fluctuation_N푸드코트.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/중분류작업/test/test_fluctuation_가공식품.csv',
 

In [ ]:
# 정규표현식 이용하여 중분류명 추출
import re
p = re.compile('(?<=test_fluctuation_).+(?=.csv)')
m = re.search(p, str(file_name_list[1]))
m.group()

'N냉장류'

In [ ]:
basedf2 = pd.read_csv('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/dataset/create_feature/train_2_to_7_짱제.csv')[['고객번호']]

In [ ]:
# 고객번호 데이터프레임에 중분류별 증감지수 열 추가

for file_name in file_name_list:
  df = pd.read_csv(file_name)
  m = re.search(p, file_name)
  name = 'f_' + m.group()
  merge_df = pd.merge(basedf1, df[['고객번호','fluctuation']], on='고객번호', how='left')  
  basedf2[name] = merge_df['fluctuation']
  # print(name, merge_df.shape)
  # print(merge_df.isna().sum())

In [ ]:
basedf2.head()

,고객번호,f_N냉동류,f_N냉장류,f_N도서음반,f_N스낵류,f_N음료류,f_N자동차용품,f_N푸드코트,f_가공식품,f_가구,...,f_준보석시계,f_차커피,f_축산가공,f_침구수예,f_캐주얼,f_커리어,f_통신컴퓨터,f_트래디셔널,f_피혁잡화,f_화장품
0,1,0.0,0.0,NaN,0.0,0.0,NaN,0.0,-2.0,NaN,...,0.0,4.0,5.0,0.0,0.0,-4.0,0.0,-3.0,-5.0,7.0
1,2,NaN,NaN,NaN,NaN,-2.0,NaN,NaN,-1.0,NaN,...,NaN,3.0,0.0,-4.0,-7.0,-1.0,NaN,4.0,0.0,0.0
2,3,0.0,0.0,NaN,1.0,-1.0,NaN,NaN,1.0,NaN,...,NaN,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,4,8.0,6.0,NaN,0.0,-2.0,NaN,NaN,-1.0,-4.0,...,NaN,2.0,-1.0,-5.0,0.0,1.0,NaN,NaN,NaN,0.0
4,5,NaN,NaN,NaN,0.0,NaN,NaN,NaN,5.0,NaN,...,NaN,7.0,9.0,0.0,2.0,NaN,NaN,0.0,NaN,5.0


In [ ]:
basedf2.to_excel('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/dataset/create_feature/test_fluctuation_중분류별.xlsx',
                 encoding='utf-8-sig', index=False)

###코딩과정

In [ ]:
mom_df = npurch.copy()

In [ ]:
df = mom_df[mom_df['중분류명'] == li[0]]
df.head()

,제휴사,고객번호,구매일자,구매시간,구매금액,중분류명,점포,거주지역,거주지,점포지역,월,년도,4분기
0,B,17218,2014-02-22,20,2420,N음료류,B44,100.0,경기,경기,2,2014,1
7,B,17218,2015-06-19,21,2500,N음료류,B44,100.0,경기,경기,6,2015,2
16,B,17218,2015-05-30,16,910,N음료류,B44,100.0,경기,경기,5,2015,2
22,B,17218,2014-06-11,18,760,N음료류,B44,100.0,경기,경기,6,2014,2
33,B,17218,2014-03-21,18,380,N음료류,B44,100.0,경기,경기,3,2014,1


In [ ]:
df = df[df['구매일자'].between('2014-01-01', '2014-03-31')]
df.shape

(103362, 13)

In [ ]:
df = df.groupby(['고객번호'], as_index=False)[['구매금액']].sum()
df.head()

,고객번호,구매금액
0,3,23100
1,6,115400
2,9,2440
3,10,8420
4,11,18520


In [ ]:
p_li = [0.04, 0.11, 0.23, 0.40, 0.60, 0.77, 0.89, 0.96] # 퍼센트
bins = [0] + [df['구매금액'].quantile(q=p_li[i]) for i in range(8)] + [df['구매금액'].quantile(1)] # 등급별 기준값
labels = [1, 2, 3, 4, 5, 6, 7, 8, 9] # 등급
df['grade14_1'] = pd.cut(df['구매금액'], bins=bins, labels=labels)
df['grade14_1'] = pd.to_numeric(df['grade14_1'])

In [ ]:
df.head()

,고객번호,구매금액,grade14_1
0,3,23100,6
1,6,115400,9
2,9,2440,2
3,10,8420,4
4,11,18520,5


In [ ]:
df.drop('구매금액', axis=1, inplace=True)
df.head()

,고객번호,grade14_1
0,3,6
1,6,9
2,9,2
3,10,4
4,11,5


##피처2. 고객 분기별 총구매금액 등급 -> 증감지수

###14년1분기~15년2분기 작업

In [ ]:
temp = npurch[npurch['구매일자'].between('2014-01-01', '2015-06-30')]
mom_df = temp.drop(['제휴사', '구매시간', '중분류명', '점포',
                     '거주지역', '거주지', '점포지역', '월', '년도', '4분기'], axis=1)

In [ ]:
df_final = pd.merge(mom_df, quan_grade(mom_df, '2014-01-01', '2014-03-31', 'grade14_1'), on='고객번호', how='left')
df_final = pd.merge(df_final, quan_grade(mom_df, '2014-04-01', '2014-06-30', 'grade14_2'), on='고객번호', how='left')
df_final = pd.merge(df_final, quan_grade(mom_df, '2014-07-01', '2014-09-30', 'grade14_3'), on='고객번호', how='left')
df_final = pd.merge(df_final, quan_grade(mom_df, '2014-10-01', '2014-12-31', 'grade14_4'), on='고객번호', how='left')
df_final = pd.merge(df_final, quan_grade(mom_df, '2015-01-01', '2015-03-31', 'grade15_1'), on='고객번호', how='left')
df_final = pd.merge(df_final, quan_grade(mom_df, '2015-04-01', '2015-06-30', 'grade15_2'), on='고객번호', how='left')

In [ ]:
df_final.iloc[:, 3:] = df_final.iloc[:, 3:].fillna(0)
# df_final.iloc[:, 3:]

In [ ]:
df_final['fluctuation'] = (df_final['grade14_1'] - df_final['grade14_2']) + \
                          (df_final['grade14_2'] - df_final['grade14_3']) + \
                          (df_final['grade14_3'] - df_final['grade14_4']) + \
                          (df_final['grade14_4'] - df_final['grade15_1']) + \
                          (df_final['grade15_1'] - df_final['grade15_2'])

In [ ]:
grouped = df_final.groupby(['고객번호', 'fluctuation'], as_index=False)[['구매금액']].sum()

In [ ]:
# df_final = pd.merge(basedf1, df, on='고객번호', how='left')[['고객번호', 'fluctuation']]
# df_final.shape

(19372, 2)

In [ ]:
# 파일 저장
grouped.to_csv('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/dataset/create_feature/train_fluctuation_total.csv',
                index=False)


###14년2분기~15년3분기 작업

merge 의 how 파라미터... 모두 right로 해서 생긴 문제가 있다. 원데이터를 기준으로 하지 않아서 row 데이터가 빠지기도 했다.

그래서 다시 원데이터를 기준으로 how='left'로 돌리니 데이터 소실 없이 결과를 출력할 수 있었다.

In [ ]:
temp = npurch[npurch['구매일자'].between('2014-04-01', '2015-09-30')]
mom_df = temp.drop(['제휴사', '구매시간', '중분류명', '점포',
                     '거주지역', '거주지', '점포지역', '월', '년도', '4분기'], axis=1)

In [ ]:
mom_df.shape

(21923576, 3)

In [ ]:
df_final = pd.merge(mom_df, quan_grade(mom_df, '2014-04-01', '2014-06-30', 'grade14_2'), on='고객번호', how='left')
df_final = pd.merge(df_final, quan_grade(mom_df, '2014-07-01', '2014-09-30', 'grade14_3'), on='고객번호', how='left')
df_final = pd.merge(df_final, quan_grade(mom_df, '2014-10-01', '2014-12-31', 'grade14_4'), on='고객번호', how='left')
df_final = pd.merge(df_final, quan_grade(mom_df, '2015-01-01', '2015-03-31', 'grade15_1'), on='고객번호', how='left')
df_final = pd.merge(df_final, quan_grade(mom_df, '2015-04-01', '2015-06-30', 'grade15_2'), on='고객번호', how='left')
df_final = pd.merge(df_final, quan_grade(mom_df, '2015-07-01', '2015-09-30', 'grade15_3'), on='고객번호', how='left')

In [ ]:
df_final.iloc[:, 3:] = df_final.iloc[:, 3:].fillna(0)
# df_final.iloc[:, 3:]

In [ ]:
df_final['fluctuation'] = (df_final['grade14_2'] - df_final['grade14_3']) + \
                          (df_final['grade14_3'] - df_final['grade14_4']) + \
                          (df_final['grade14_4'] - df_final['grade15_1']) + \
                          (df_final['grade15_1'] - df_final['grade15_2']) + \
                          (df_final['grade15_2'] - df_final['grade15_3'])

In [ ]:
print(df_final.shape)
df_final.head()

(21923576, 10)


,고객번호,구매일자,구매금액,grade14_2,grade14_3,grade14_4,grade15_1,grade15_2,grade15_3,fluctuation
0,17218,2014-12-09,3440,3.0,5.0,3.0,5.0,4.0,5.0,-2.0
1,17218,2014-12-09,5480,3.0,5.0,3.0,5.0,4.0,5.0,-2.0
2,17218,2015-06-19,2500,3.0,5.0,3.0,5.0,4.0,5.0,-2.0
3,17218,2015-06-19,4220,3.0,5.0,3.0,5.0,4.0,5.0,-2.0
4,17218,2015-06-19,7560,3.0,5.0,3.0,5.0,4.0,5.0,-2.0


In [ ]:
grouped = df_final.groupby(['고객번호', 'fluctuation'], as_index=False)[['구매금액']].sum()
grouped[:10]

,고객번호,fluctuation,구매금액
0,1,1.0,70605022
1,2,0.0,73756070
2,3,1.0,2604284
3,4,1.0,13774022
4,5,5.0,7475480
5,6,0.0,19136522
6,7,0.0,48118753
7,8,0.0,45735210
8,9,0.0,169850660
9,10,1.0,14253129


In [ ]:
grouped.shape

(19371, 3)

In [ ]:
# df_final = pd.merge(basedf2, grouped, on='고객번호', how='left')[['고객번호', 'fluctuation']]
# df_final.shape

(19371, 2)

In [ ]:
# 파일 저장
grouped.to_csv('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/dataset/create_feature/test_fluctuation_total.csv',
                index=False)


##피처3. 고급/일반

- 고급

In [ ]:
fname = '내구'
# df = pd.read_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./{fname}.csv')
df = df.set_index('고객번호')

# for k in range(8):
#   p_li = [0.04, 0.11, 0.23, 0.40, 0.60, 0.77, 0.89, 0.96] # 퍼센트
#   bins = [0] + [df.iloc[:, k].quantile(q=p_li[i]) for i in range(8)] + [df.iloc[:, k].quantile(1)] # 등급별 기준값
#   labels = [1, 2, 3, 4, 5, 6, 7, 8, 9] # 등급

#   df.iloc[:, k] = pd.cut(df.iloc[:, k], bins=bins, labels=labels, duplicates='raise')
#   df.iloc[:, k] = pd.to_numeric(df.iloc[:, k])

# # 전문품은 가격 차이가 크므로 5개의 등급으로 구분한다.
# for k in range(8):
#   p_li = [0.50, 0.60, 0.70, 0.80, 0.90] # 퍼센트
#   bins = [0] + [df.iloc[:, k].quantile(q=p_li[i]) for i in range(4)] + [df.iloc[:, k].quantile(1)] # 등급별 기준값
#   labels = [1, 2, 3, 4, 5] # 등급

# # 전문품은 가격 차이가 크므로 5개의 등급으로 구분한다.
for k in range(8):
  p_li = [0.23, 0.40, 0.60, 0.77, 0.89, 0.96] # 퍼센트
  bins = [0] + [df.iloc[:, k].quantile(q=p_li[i]) for i in range(5)] + [df.iloc[:, k].quantile(1)] # 등급별 기준값
  labels = [1, 2, 3, 4, 5, 6] # 등급

  df.iloc[:, k] = pd.cut(df.iloc[:, k], bins=bins, labels=labels, duplicates='raise')
  df.iloc[:, k] = pd.to_numeric(df.iloc[:, k])

train = df.iloc[:, :6].fillna(0)

train[fname] = (train['2014-01-01'] - train['2014-04-01']) + \
                          (train['2014-04-01'] - train['2014-07-01']) + \
                          (train['2014-07-01'] - train['2014-10-01']) + \
                          (train['2014-10-01'] - train['2015-01-01']) + \
                          (train['2015-01-01'] - train['2015-04-01'])
train = train.reset_index()
train = train[['고객번호', fname]]

test = df.iloc[:, 1:7].fillna(0)
test[fname] = (test['2014-04-01'] - test['2014-07-01']) + \
                          (test['2014-07-01'] - test['2014-10-01']) + \
                          (test['2014-10-01'] - test['2015-01-01']) + \
                          (test['2015-01-01'] - test['2015-04-01']) + \
                          (test['2015-04-01'] - test['2015-07-01']) 
test = test.reset_index()
test = test[['고객번호', fname]]

# train_merge = pd.merge(basedf1, train, on='고객번호', how='left')
# test_merge = pd.merge(basedf2, test, on='고객번호', how='left')

train_merge = pd.merge(train_merge, train, on='고객번호', how='left')
test_merge = pd.merge(test_merge, test, on='고객번호', how='left')

In [ ]:
train_merge = train_merge.fillna(0.001)
train_merge.head()

,고객번호,전문,일반,준내구,비내구,고급,선매,편의,내구
0,1,1.000,0.0,0.0,2.0,1.0,0.0,-1.0,1.0
1,2,5.000,4.0,0.0,0.0,0.0,0.0,3.0,0.0
2,3,0.001,0.0,0.0,0.0,1.0,0.0,1.0,-2.0
3,4,-3.000,-1.0,0.0,-1.0,0.0,0.0,-1.0,0.0
4,5,-1.000,5.0,4.0,4.0,2.0,5.0,6.0,4.0


In [ ]:
test_merge = test_merge.fillna(0.001)
test_merge.head()

,고객번호,전문,일반,준내구,비내구,고급,선매,편의,내구
0,1,0.000,1.0,0.0,6.0,0.0,0.0,3.0,0.0
1,2,1.000,3.0,-1.0,0.0,0.0,-1.0,3.0,0.0
2,3,0.001,0.0,0.0,1.0,1.0,2.0,0.0,-1.0
3,4,1.000,1.0,0.0,1.0,0.0,0.0,2.0,-1.0
4,5,4.000,5.0,5.0,7.0,6.0,5.0,7.0,4.0


In [ ]:
# train_merge.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./train_append.csv')
# test_merge.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./test_append.csv')

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./전문.csv')
df = df.set_index('고객번호')
df.head()

,2014-01-01,2014-04-01,2014-07-01,2014-10-01,2015-01-01,2015-04-01,2015-07-01,2015-10-01
고객번호,,,,,,,,
1,19467.222222,21009.560440,5779.347826,293.478261,7816.666667,2782.417582,5183.695652,1911.956522
2,10002.666667,8340.109890,1395.000000,10197.934783,58899.888889,0.000000,1454.347826,858.695652
4,0.000000,8098.901099,271.739130,0.000000,0.000000,531.868132,2173.913043,1092.391304
5,2628.333333,2010.109890,2671.739130,1086.956522,4476.555556,6148.021978,0.000000,6593.478261
6,1248.888889,1076.923077,4646.739130,1771.739130,655.555556,4219.780220,2434.782609,3857.608696


In [ ]:
# 전문품은 가격 차이가 크므로 5개의 등급으로 구분한다.
for k in range(8):
  p_li = [0.50, 0.60, 0.70, 0.80, 0.90] # 퍼센트
  bins = [0] + [df.iloc[:, k].quantile(q=p_li[i]) for i in range(4)] + [df.iloc[:, k].quantile(1)] # 등급별 기준값
  labels = [1, 2, 3, 4, 5] # 등급

  df.iloc[:, k] = pd.cut(df.iloc[:, k], bins=bins, labels=labels, duplicates='raise')
  df.iloc[:, k] = pd.to_numeric(df.iloc[:, k])
df.head()

,2014-01-01,2014-04-01,2014-07-01,2014-10-01,2015-01-01,2015-04-01,2015-07-01,2015-10-01
고객번호,,,,,,,,
1,5.0,5.0,5.0,2.0,5.0,4.0,5.0,4.0
2,5.0,5.0,4.0,5.0,5.0,NaN,4.0,3.0
4,NaN,5.0,2.0,NaN,NaN,3.0,4.0,3.0
5,4.0,4.0,4.0,3.0,5.0,5.0,NaN,5.0
6,4.0,3.0,5.0,4.0,3.0,5.0,4.0,4.0


In [ ]:
'''for k in range(8):
  p_li = [0.04, 0.11, 0.23, 0.40, 0.60, 0.77, 0.89, 0.96] # 퍼센트
  bins = [0] + [df.iloc[:, k].quantile(q=p_li[i]) for i in range(8)] + [df.iloc[:, k].quantile(1)] # 등급별 기준값
  labels = [1, 2, 3, 4, 5, 6, 7, 8, 9] # 등급

  df.iloc[:, k] = pd.cut(df.iloc[:, k], bins=bins, labels=labels, duplicates='raise')
  df.iloc[:, k] = pd.to_numeric(df.iloc[:, k])
df.head()'''

In [ ]:
train = df.iloc[:, :6].fillna(0)

train['fluctuation'] = (train['2014-01-01'] - train['2014-04-01']) + \
                          (train['2014-04-01'] - train['2014-07-01']) + \
                          (train['2014-07-01'] - train['2014-10-01']) + \
                          (train['2014-10-01'] - train['2015-01-01']) + \
                          (train['2015-01-01'] - train['2015-04-01'])
train = train.reset_index()
train.head()

,고객번호,2014-01-01,2014-04-01,2014-07-01,2014-10-01,2015-01-01,2015-04-01,fluctuation
0,1,5.0,5.0,5.0,2.0,5.0,4.0,1.0
1,2,5.0,5.0,4.0,5.0,5.0,0.0,5.0
2,4,0.0,5.0,2.0,0.0,0.0,3.0,-3.0
3,5,4.0,4.0,4.0,3.0,5.0,5.0,-1.0
4,6,4.0,3.0,5.0,4.0,3.0,5.0,-1.0


In [ ]:
test = df.iloc[:, 1:7].fillna(0)
test['fluctuation'] = (test['2014-04-01'] - test['2014-07-01']) + \
                          (test['2014-07-01'] - test['2014-10-01']) + \
                          (test['2014-10-01'] - test['2015-01-01']) + \
                          (test['2015-01-01'] - test['2015-04-01']) + \
                          (test['2015-04-01'] - test['2015-07-01']) 
test = test.reset_index()
test.head()

,고객번호,2014-04-01,2014-07-01,2014-10-01,2015-01-01,2015-04-01,2015-07-01,fluctuation
0,1,5.0,5.0,2.0,5.0,4.0,5.0,0.0
1,2,5.0,4.0,5.0,5.0,0.0,4.0,1.0
2,4,5.0,2.0,0.0,0.0,3.0,4.0,1.0
3,5,4.0,4.0,3.0,5.0,5.0,0.0,4.0
4,6,3.0,5.0,4.0,3.0,5.0,4.0,-1.0


In [ ]:
# train[['고객번호', 'fluctuation']].to_csv('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./train_fluctuation_전문.csv')
# test[['고객번호', 'fluctuation']].to_csv('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./test_fluctuation_전문.csv')

##피처4. 선매/편의...

##(버림) 피처1. 고객의 중분류상품 구매금액/ 중분류상품 총판매금액


In [ ]:
train_purch = npurch[npurch['구매일자'].between('2015-01-01', '2015-06-30')]
train_purch.head()

,제휴사,고객번호,구매일자,구매시간,구매금액,중분류명,점포,거주지역,거주지,점포지역,월,년도,4분기
0,B,17218,2014-02-22,20,2420,N음료류,B44,100.0,경기,경기,2,2014,1
1,B,17218,2014-02-22,20,1070,주류,B44,100.0,경기,경기,2,2014,1
2,B,17218,2014-02-22,20,8060,주류,B44,100.0,경기,경기,2,2014,1
3,B,17218,2014-02-22,20,6000,일용잡화,B44,100.0,경기,경기,2,2014,1
5,B,17218,2014-12-09,19,3440,N스낵류,B44,100.0,경기,경기,12,2014,4


In [ ]:
# 각 고객의 중분류별 총구매금액
grouped1 = train_purch.groupby(['고객번호', '중분류명'], as_index=False)[['구매금액']].sum()
grouped1.head()

,고객번호,중분류명,구매금액
0,1,N냉동류,13960
1,1,N냉장류,43120
2,1,N스낵류,32300
3,1,N음료류,37560
4,1,N푸드코트,8000


In [ ]:
# 중분류별 총매출
grouped2 = train_purch.groupby(['중분류명'])[['구매금액']].sum()
grouped2.head()

,구매금액
중분류명,
N냉동류,2502559104
N냉장류,1932757205
N도서/음반,1497169045
N스낵류,6185018005
N음료류,3033844720


In [ ]:
# merge 로 합치기
df = pd.merge(grouped1, grouped2, on='중분류명', how='left')

# 총매출 대비 고객의 총구매액
df['매출대비구매비율'] = df['구매금액_x'] / df['구매금액_y']

In [ ]:
df.head()

,고객번호,중분류명,구매금액_x,구매금액_y,매출대비구매비율
0,1,N냉동류,13960,2502559104,0.000006
1,1,N냉장류,43120,1932757205,0.000022
2,1,N스낵류,32300,6185018005,0.000005
3,1,N음료류,37560,3033844720,0.000012
4,1,N푸드코트,8000,962241397,0.000008


In [ ]:
df = df.pivot(index = '고객번호', columns = '중분류명', values='매출대비구매비율').fillna(0)
df.head()

중분류명,N냉동류,N냉장류,N도서/음반,N스낵류,N음료류,N자동차용품,N푸드코트,가공식품,가구,건강식품,...,준보석/시계,차/커피,축산가공,침구/수예,캐주얼,커리어,통신/컴퓨터,트래디셔널,피혁잡화,화장품
고객번호,,,,,,,,,,,,,,,,,,,,,
1,0.000006,0.000022,0.0,0.000005,0.000012,0.0,0.000008,0.000066,0.0,0.000452,...,0.000259,0.000056,0.000166,0.000924,0.000196,0.000257,0.000026,0.000248,0.000096,1.057061e-04
2,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000044,0.0,0.000033,...,0.000000,0.000097,0.000374,0.000108,0.000006,0.000033,0.000000,0.000092,0.000040,5.897167e-04
3,0.000002,0.000043,0.0,0.000007,0.000052,0.0,0.000000,0.000012,0.0,0.000000,...,0.000000,0.000019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.723393e-07
4,0.000055,0.000023,0.0,0.000001,0.000003,0.0,0.000000,0.000028,0.0,0.000011,...,0.000000,0.000032,0.000107,0.000071,0.000015,0.000067,0.000000,0.000000,0.000028,2.533276e-05
5,0.000000,0.000000,0.0,0.000002,0.000000,0.0,0.000000,0.000029,0.0,0.000253,...,0.000000,0.000044,0.001104,0.000036,0.000002,0.000000,0.000000,0.000013,0.000000,4.973731e-05


In [ ]:
df.columns

Index(['N냉동류', 'N냉장류', 'N도서/음반', 'N스낵류', 'N음료류', 'N자동차용품', 'N푸드코트', '가공식품',
       '가구', '건강식품', '골프용품', '교복', '구두', '기타', '기타의류', '남성 트랜디', '남성정장',
       '남성캐주얼', '농산물', '대형가전', '디자이너', '란제리/내의', '명품', '모피/피혁', '문구/팬시', '보석',
       '생활잡화', '섬유잡화', '셔츠', '소형가전', '수산품', '스포츠', '시티웨어', '식기', '아동', '악기',
       '액세서리', '웨딩', '육류', '일용잡화', '전자/전기', '젓갈/반찬', '주류', '주방가전', '주방용품',
       '준보석/시계', '차/커피', '축산가공', '침구/수예', '캐주얼', '커리어', '통신/컴퓨터', '트래디셔널',
       '피혁잡화', '화장품'],
      dtype='object', name='중분류명')

In [ ]:
len(df.columns)

55

In [ ]:
# df.to_csv('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/작업결과물/etc./매출대비구매비율.csv')